In [ ]:
import torch
import torch.nn as nn
import pickle
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'



vocab_size = 5520  
n_embd = 768
block_size = 32
n_layer = 8
n_head = 8
dropout = 0.2

class Head(nn.Module):
    

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
       
        B,T,C = x.shape
        k = self.key(x)  
        q = self.query(x) 
      
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
      
        v = self.value(x) 
        out = wei @ v 
        return out

class MultiHeadAttention(nn.Module):
  

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out
    

class FeedFoward(nn.Module):
    

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
   

    def __init__(self, n_embd, n_head):
       
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x
    
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        
        
     
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb 
        x = self.blocks(x) 
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            index_cond = index[:, -block_size:]
            
            logits, loss = self.forward(index_cond)
           
            logits = logits[:, -1, :] 
           
            probs = F.softmax(logits, dim=-1) 
            
            index_next = torch.multinomial(probs, num_samples=1) 
           
            index = torch.cat((index, index_next), dim=1)
        return index






model_loaded = GPTLanguageModel(vocab_size)
model_loaded.load_state_dict(torch.load("gpt_model_weights.pth", map_location='cuda' if torch.cuda.is_available() else 'cpu'))

print(device)
print("Reloaded model with weights from state_dict")


In [ ]:


model_loaded.to(device)
model_loaded.eval()


dummy_input = torch.randint(0, vocab_size, (1, 1), dtype=torch.long).to(device)

# Export the model
torch.onnx.export(
    model_loaded,             
    dummy_input,               
    "gpt_model.onnx",           
    input_names=["input_ids"],  
    output_names=["logits"],   
    dynamic_axes={
        "input_ids": {1: "sequence_length"}, 
        "logits": {1: "sequence_length"}
    },
    opset_version=13,
    do_constant_folding=True
)

print(" Model exported to ONNX successfully: gpt_model.onnx")
